In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from IPython.display import display, Image
from ipywidgets import FileUpload

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


96112376/96112376 [==============================] - 1s 0us/step


In [ ]:
# Define categories
categories = os.listdir('/content/drive/MyDrive/food20dataset/test_set')
img_size = 299

# Load the InceptionV3 model without the top classification layer
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

# Add your custom classification layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Preprocess and predict on the uploaded image
def predict_image_class(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return categories[predicted_class]

# Load and preprocess the image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_size, img_size))
    img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
    img = np.expand_dims(img, axis=0)
    return img

# Create an interactive file uploader widget
uploader = FileUpload(accept='.jpg', multiple=False)

# Display the uploader widget
display(uploader)

# Function to handle image upload and prediction
def handle_upload(change):
    uploaded_filename = next(iter(uploader.value))
    image_data = uploader.value[uploaded_filename]['content']
    image_path = '/content/' + uploaded_filename

    # Save the uploaded image
    with open(image_path, 'wb') as f:
        f.write(image_data)

    # Display the uploaded image
    display(Image(filename=image_path, width=300, height=300))

    # Make a prediction
    predicted_class = predict_image_class(image_path)
    print("Predicted class:", predicted_class)

# Attach the event handler to the uploader
uploader.observe(handle_upload, names='value')


In [ ]:
import os
import cv2
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop  # Import RMSprop optimizer
from IPython.display import display, Image
from ipywidgets import FileUpload

In [ ]:
# Define categories
categories = os.listdir('/content/drive/MyDrive/food20dataset/test_set')
num_classes = len(categories)

# Load and preprocess the dataset
img_size = 299

In [ ]:
def preprocess_data(data_dir):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append(resized_arr)
                labels.append(class_num)
            except Exception as e:
                print(e)
    return np.array(data), np.array(labels)

x_data, y_labels = preprocess_data('/content/drive/MyDrive/food20dataset/train_set')

# Split the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_data, y_labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

In [ ]:
# Load the InceptionV3 model without the top classification layer
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))


In [ ]:
# Load the InceptionV3 model without the top classification layer
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

# Add your custom classification layers on top of InceptionV3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with RMSprop optimizer
model.compile(optimizer=RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                             preprocessing_function=preprocess_input)

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Hyperparameter Tuning
learning_rates = [0.001, 0.0001]
batch_size = 32
epochs = 20

for lr in learning_rates:
    print(f"Learning Rate: {lr}, Optimizer: RMSprop")

    # Compile the model with the current learning rate
    model.compile(optimizer=RMSprop(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=len(x_train) // batch_size,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        callbacks=[early_stopping])

Learning Rate: 0.001, Optimizer: RMSprop
Epoch 1/20
35/35 [==============================] - 1328s 37s/step - loss: 3.1265 - accuracy: 0.0598 - val_loss: 14315655.0000 - val_accuracy: 0.0493
Epoch 2/20
35/35 [==============================] - 1315s 38s/step - loss: 3.0247 - accuracy: 0.0526 - val_loss: 694469.7500 - val_accuracy: 0.0704
Epoch 3/20
35/35 [==============================] - 1301s 37s/step - loss: 3.0054 - accuracy: 0.0553 - val_loss: 964.6967 - val_accuracy: 0.0493
Epoch 4/20
35/35 [==============================] - 1307s 37s/step - loss: 3.0170 - accuracy: 0.0490 - val_loss: 4662.7168 - val_accuracy: 0.0493
Epoch 5/20
35/35 [==============================] - 1292s 37s/step - loss: 3.0089 - accuracy: 0.0499 - val_loss: 1232.8551 - val_accuracy: 0.0493
Epoch 6/20
35/35 [==============================] - 1301s 37s/step - loss: 3.0028 - accuracy: 0.0526 - val_loss: 12429.8828 - val_accuracy: 0.0493
Epoch 7/20
35/35 [==============================] - 1303s 37s/step - loss: 3.

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Food_Classification/inceptionv3_model.hdf5')

In [ ]:
# Load the trained model
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Food_Classification/inceptionv3_model.hdf5')

# Create an interactive file uploader widget
uploader = FileUpload(accept='.jpg', multiple=False)

# Display the uploader widget
display(uploader)

# Preprocess and predict on the uploaded image
def preprocess_image(image_data):
    img = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
    img = cv2.resize(img, (img_size, img_size))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to handle image upload and prediction
def handle_upload(change):
    uploaded_filename = next(iter(uploader.value))
    image_data = uploader.value[uploaded_filename]['content']

    # Preprocess the uploaded image
    img = preprocess_image(image_data)

    # Make a prediction
    prediction = model.predict(img)
    predicted_class